Deep Learning Final Project

Generate IoU, Dice, Sensitivity, Specificity Metrics from Model Results/PNGs

In [1]:
# input masks in a mask zip file, and predictions in a prediction zip file
# items are, say, mask/---.png and prediction/---.png

In [2]:
!unzip '*.zip'

unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.


In [3]:
import fnmatch
import os

indexes=[]
for file in os.listdir('./mask'):
  if fnmatch.fnmatch(file, 'brats_val_*_t1_123_w_mask_*.png'):
    if indexes.count(int(file[10:13])) == 0:
      indexes.append(int(file[10:13]))
indexes=sorted(indexes)
print(indexes)

FileNotFoundError: [Errno 2] No such file or directory: './mask'

In [ ]:
import imageio.v3 as iio
import numpy as np

im = iio.imread('download.png')
imin2 = iio.imread('download (1).png')
mask=(imin2[:,:,0]>200).astype(np.uint8)
invmask=1-mask
im0=im[:,:,0]
im1=im[:,:,1]
im2=im[:,:,2]
im3=im[:,:,3]

In [ ]:
def IoU(mask,img):
  falseP=(1-mask)*(img/255.0)
  trueP=mask*(img/255.0)
  return sum(sum(trueP))/(sum(sum(falseP))+sum(sum(mask.astype(int))))

def sensitivity(mask,img):
  return sum(sum(img/255.0))/sum(sum(mask.astype(int)))

In [ ]:
IoUsum = 0
positsum = 0
predpsum = 0
upredsum = 0
opredsum = 0
total = 0
TNsum = 0
for i in indexes:
  for l in [0,1,2,3,4]:
    pred = iio.imread('prediction/brats_val_' + str(i) + '_t1_123_w_pred_' + str(l) + '.png')
    pred = pred[:,:,0]/3+pred[:,:,1]/3+pred[:,:,2]/3
    mask = iio.imread('mask/brats_val_' + str(i) + '_t1_123_w_mask_' + str(l) + '.png')
    mask = mask[:,:,0]/3+mask[:,:,1]/3+mask[:,:,2]/3
    mask=(mask[:,:]>200).astype(np.uint8)
    print(str(i) + "." + str(l) + ": IoU: " + str(IoU(mask,pred)) + " sens: " + str(sensitivity(mask,pred)))
    total = total + 1
    IoUsum = IoUsum + IoU(mask,pred)
    positsum = positsum + sum(sum(mask.astype(int)))
    predpsum = predpsum + sum(sum(pred/255.0))
    upredsum = upredsum + sum(sum(mask*(pred/255.0)))
    opredsum = opredsum + sum(sum((1-mask)*(pred/255.0)))
    TNsum = TNsum + sum(sum((1-mask)*(255-pred)/255))
print("IoU: " + str(upredsum/(opredsum+positsum)))
print("sensitivity: " + str(predpsum/positsum))
print("specificity: " + str(TNsum/(TNsum+opredsum)))
print("Dice: " + str(2*upredsum/(predpsum+positsum)))